<a href="https://colab.research.google.com/github/mongbleu/proejct4/blob/main/220523_efficient.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install IPython

In [ ]:
#! pip install tensorflow==2.9.0

In [ ]:
! tensorflow --version

In [ ]:
#필요한 모듈 불러오기
import tensorflow as tf
import numpy as np
import os

import matplotlib.pyplot as plt
from IPython.display import clear_output
from PIL import Image
from tqdm.auto import trange

from keras import layers
from tensorflow.keras.utils import load_img, img_to_array

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
#해당 디렉토리 경로에 있는 파일들의 이름 가져오기
root_dir="/content/drive/MyDrive/bird-classification/train"

In [ ]:
img_path_list = []
root = os.walk(root_dir)
for image_dir in root:
  img_path_list.append(image_dir)

In [ ]:
image_dirs = img_path_list[1:]

In [ ]:
len(image_dirs)

In [ ]:
dir_names = img_path_list[0][1]
CLASS_NUM = len(dir_names)
CLASS_NUM

### 데이터 불러오기 2차 시도

In [ ]:
real_path = []
for i in range(0, len(image_dirs)):
  for j in range(0, len(image_dirs[i][2])):
    real_path.append(image_dirs[i][0]+ '/' + image_dirs[i][2][j])

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [ ]:
#이미지를 하나 받아 preprocessing하기

def load_preprocessing(file_dir):
    #Normalization
    input_image = Image.open(file_dir)
    input_image = img_to_array(input_image)
    input_image = tf.cast(input_image, tf.float32) / 255.0 #-1 ~ 1로도 해보기

    #resize
    input_image = tf.image.resize(input_image, (28, 28))
    #input_image = layers.experimental.preprocessing.Resizing(224,224)(input_image)

    #print(f'min_max_pixel : {np.min(input_image), np.max(input_image)}')

    return input_image

In [ ]:
#change all image into array
dataset = np.array([load_preprocessing(image) for image in real_path])

In [ ]:
type(dataset)

In [ ]:
import tensorflow.keras.backend as K
dataset_t = K.constant(dataset)

In [ ]:
! pip install -q tf-nightly

In [ ]:
#sequential로 선언하는 data augmentation

#fildp & rotate
def data_augmentation_1(input_image):
  input_image = tf.image.filp_left_right(input_image)
  input_image = tf.keras.preprocessing.image.random_rotation(input_image,rg=0.4)    
  return input_image     

#rotate & zoom
def data_augmentation_2(input_image):
  input_image = tf.keras.preprocessing.image.random_rotation(input_image, rg=0.5),
  input_image = tf.keras.preprocessing.image.random_zoom(input_image, 0.3, 0.3)
  return input_image

#translate & brightness
def data_augmentation_3(input_image):
  input_image = tf.keras.preprocessing.random_channel_shift(input_image, 100),
  input_image = tf.keras.preprocessing.image.random_brightness(input_image, factor=0.3)
  return input_image

In [ ]:
#train데이터셋 로드하기, map함수로 로드..?
#map funciton : tf.Tensor -> tf.Tensor (pandas apply와 유사)

#@tf.funciton (debugging 다 하고 가져오기)
def load_train_image(input_image):
  #image_augmentation
  if tf.random.uniform(()) > 0.5:
    input_image = data_augmentation_1(input_image)
  elif tf.random.uniform(()) > 0.5:
    input_image = data_augmentation_2(input_image)
  elif tf.random.uniform(())<0.5:
    input_image = data_augmentation_3(input_image)

  #normalize
  input_image = input_image / 225.0
  #input_image = normalize(input_image)
  #input_image = layers.experimental.preprocessing.Rescaling(1./255)(input_image)
  #input_image = layers.experimental.preprocesisng.Rescaling(1./127.5, offset=-1)(input_image)
  return input_image

In [ ]:
os.getcwd()

### Dataset 불러오기 2차 시도
- generator data augmentation 시도

In [ ]:
#dataset retry
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(validation_split=0.2,
                  rescale = 1./255)

train_data_gen = image_generator.flow_from_directory(
  root_dir,
  subset='training',
  target_size=(244,244),
  batch_size=256,
  class_mode='sparse',
  seed=123)

test_data_gen = image_generator.flow_from_directory(
    root_dir,
    subset='validation',
    target_size=(244,244),
    class_mode='sparse',
    batch_size=256,
    seed=123)

In [ ]:
len(train_data_gen)

In [ ]:
#for image_batch in train_data_gen:
  #print(image_batch.shape)
  #break

In [ ]:
#norm_ds = train_ds.map(lambda x : 
                      # (tf.keras.layers.experimental.preprocessing.Rescaling(1./255)(x)))
#image_batch = next(iter(norm_ds))

In [ ]:
first_image = image_batch[0]
np.min(first_image), np.max(first_image)

In [ ]:
#label 만들기 : train/test/val의 class가 모두 다른 것을 발견..!
#train label
#dataset_labels = [dir.split('/')[6] for dir in real_path]

In [ ]:
#len(set(dataset_labels))

In [ ]:
#label encoding
"""from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit_transform(dataset_labels)
print(f'label_num : {len(label_encoder.classes_)}')
label_encoder.classes_"""

In [ ]:
"""label_encoder.transform(dataset_labels)"""

In [ ]:
"""#train-test split
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(train_ds, dataset_labels, test_size=0.2, random_state=42)"""

In [ ]:
"""len(x_train), len(x_test), len(y_train), len(y_test)"""

## 모델 만들기

###최적화

In [ ]:
from tensorflow.keras.regularizers import l1, l2

In [ ]:
#early stopping

#파라미터 저장경로
checkpoint_filepath = 'FMbest.hdf5' #FMbest는 이름, hdf5는 확장자명

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1)

save_best = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath, monitor='val_loss', verbose=1, save_best_only=True,
    save_weights_only=True, mode='auto', save_freq='epoch', options=None)

### 데이터 준비

In [ ]:
from keras import Sequential
from keras import layers
from tensorflow.keras.applications import EfficientNetB0, EfficientNetB4, EfficientNetB7
from tensorflow.keras.applications import ResNet50

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
#CLASS_NUM
TRAIN_LENGTH = 23309
BATCH_SIZE = 256
BUFFER_SIZE = 23309
STEPS_PER_EPOCH = TRAIN_LENGTH // BATCH_SIZE #버림으로 계산

In [ ]:
"""#데이터 섞기 : x, y를 같이 섞어야함..!ㅠ
#cache() : 데이터셋을 메모리 또는 로컬 저장소에 캐시. 각 에포크 동안 실행되는 일부 작업이 저장된다
x_train = x_train.cache().shuffle().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
#연산에 필요한 함수를 미리가져오는 함수 / #데이터를 담는 그릇의 크기를 동적으로 조정한다
x_train = x_train.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
x_test = x_test.bath(BATCH_SIZE)"""

### ResNet50

In [ ]:
#Resnet basic
#input따로 만들기
input = layers.Input(shape=(244, 244, 3))
model = ResNet50(
    include_top=False,
    weights = 'imagenet',
    input_shape = (244,244,3)
)

model.trainable = False

#output
output = model(input)
output = layers.GlobalAveragePooling2D()(output)
output = layers.Dense(units=128, activation='relu')(output)
output = layers.Dense(units=CLASS_NUM, activation='softmax')(output)

#compile
model_resnet50 = tf.keras.Model(inputs=input, outputs=output)
model_resnet50.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_resnet50.summary()

In [ ]:
model_resnet50.fit(train_data_gen, batch_size=BATCH_SIZE, epochs=10, 
          validation_data=0.2, callbacks=[early_stop, save_best])

In [ ]:
score, acc = model.evaluate(test_data_gen, batch_size=BATCH_SIZE, epochs=10)
print(f'test_score : {score}\ntest_accuracy : {acc}')

### Efficient model

In [ ]:
#efficient 모델 : 출력층만 다시 설계 (early stopping 빼고 아무런 최적화방법 사용하지 않음)

#input
input = layers.Input(shape=(244, 244,3))
#출력층 다시 만들기
efficient = EfficientNetB0(include_top=False,input_tensor=input, weights='imagenet')(input)
efficient.trainable = False
output = efficient(input)
output = layers.GlobalAveragePooling2D ()(output)
output = layers.Dense(units=128, activation='relu')(output)
output = layers.Dense(units=CLASS_NUM, activation='softmax')(output)

#compile
model_eff_base = tf.keras.Model(inpust=input, outputs=output)

In [ ]:
#efficientnetbo : input(244,244,3) - 출력층만 다시 설계
#input 만들기
input = tf.keras.layers.Input(shape=(244, 244, 3)) #model_effbo_no.input

#output만들기
effb0_2 = EfficientNetB0(include_top=False, input_tensor=input, weights='imagenet')
effb0_2.trainable = False

output = Sequential()
output.add(layers.GlobalAveragePooling2D())
output.add(layers.BatchNormalization())
output.add(layers.Dropout(rate=0.3, seed=42))
output.add(layers.Dense(units=32, kernel_regularizer=l1(0.01), activity_regularizer=l1(0.01), activation='relu'))
output.add(layers.Dropout(rate=0.3))
output.add(layers.Dense(units=CLASS_NUM, activation='softmax'))

#model
model_effb0_2 = tf.keras.Model(inputs=input, outputs=output)
model_effb0_2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-6, beta_1=0.89), 
                      loss='sparse_categorical_crossentropy', mertrics=['accuracy'])

In [ ]:
model_effb0_2.summary()

In [ ]:
plot_model(model_effb0_2)

In [ ]:
#efficientnetb0 : fitting
model.fit(train_data_gen, batch_size=BATCH_SIZE, epochs=10, verbose=1,
          validation_split=0.2, callbacks=[early_stop, save_best])

In [ ]:
#evaluate
score, acc = model.evaluate(test_data_gen, batch_size=BATCH_SIZE, epochs=10, verbose=1)
print(f'test_score : {score}\ntest_accuracy : {acc}')

In [ ]:
#efficientnetB4 : input_shape(380,380, 3)

In [ ]:
#efficientnetB7 : input_shape(600, 600, 3)